#### Μέλη ομάδας:
Ηλίας Δημητρακάκης sdi2000046

Βίκτωρ Λαμπρόπουλος sdi1900096

Άγγελος Καλογρίδης sdi2000062



#Οδηγίες για να περαστούν τα csv.
1. https://drive.google.com/drive/folders/1-EejCaL215EZoEIAhw4tN5jFr1WiViq2
2. Επιλογή όλων
3. Περισσότερες Ενέργειες > Οργάνωση > Προσθήκη Συντόμευσης
4. Αποθήκευση στο Όλες οι Τοποθεσίες (πάνω δεξιά) To Drive μου > TEDE


In [ ]:
#installs are separated from imports because we have to downgrade some libraries
!python3.11 -m pip install scipy==1.10.1 gensim==4.3.2 numpy xarray-einstats
!python3.11 -m pip install -q spacy datasets transformers scikit-learn
!python3.11 -m python -m spacy download en_core_web_sm
!python3.11 -m pip install kneed
!python3.11 -m pip install numpy
!python3.11 -m pip install pandas 
!python3.11 -m pip install matplotlib
!python3.11 -m pip install nltk
!python3.11 -m pip install seaborn
!python3.11 -m pip install torch

### ❗❗ **IMPORTANT: YOU MUST RESTART THE SESSION TO EXECUTE THE REST OF THE CELLS** ❗❗



*   Click on the down arrow next to Run All
*   Click Restart Session

#### helpers and misc for all upcoming cells  

In [7]:
!python3.11 -m pip install numpy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 16.3 MB/s eta 0:00:0000:0100:01


In [2]:
# Imports
import os
import re
import html
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import gensim.downloader as api
import scipy
import gensim
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from kneed import KneeLocator
from sklearn.model_selection import train_test_split, cross_validate,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import seaborn as sns
from scipy.sparse import hstack
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, pipeline
from datasets import load_dataset, Features, Value, Sequence

# from google.colab import drive

# # Mount Google Drive and setup save path
# drive.mount('/content/drive')
save_path = "TEDE/"
os.makedirs(save_path, exist_ok=True)

# NLTK downloads (quiet mode)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('punkt_tab', quiet=True)

# Print versions to confirm everything loaded correctly

print("NumPy version:", np.__version__)
print("SciPy version:", scipy.__version__)
print("Gensim version:", gensim.__version__)

# Initialize variables and objects for text processing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
label_encoder = LabelEncoder()

# Text preprocessing function (used for tasks)
def preprocess_text(text):
    if pd.isnull(text) or not isinstance(text, str):
        return ""
    text = html.unescape(text)
    text = re.sub(r'<[^>]+>', ' ', text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word.isalpha()]
    return " ".join(tokens)

# Helper function to read CSV from save_path
def read_csv(filename):
    return pd.read_csv(os.path.join(save_path, filename), low_memory=False)

# Dataset URLs
beauty="https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/raw/meta_categories/meta_All_Beauty.jsonl"
beauty_reviews="https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/raw/review_categories/All_Beauty.jsonl"

software="https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/raw/meta_categories/meta_Software.jsonl"
software_reviews="https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/raw/review_categories/Software.jsonl"

handmade_products="https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/raw/meta_categories/meta_Handmade_Products.jsonl"
handmade_products_reviews="https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/raw/review_categories/meta_Handmade_Products.jsonl"

digital_music="https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/raw/meta_categories/meta_Digital_Music.jsonl"
digital_music_reviews="https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/raw/review_categories/Digital_Music.jsonl"

appliances="https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/raw/meta_categories/meta_Appliances.jsonl"
appliances_reviews="https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/raw/review_categories_categories/meta_Appliances.jsonl"

categories=[
    'All_Beauty',
    'Software',
    'Handmade_Products',
    'Appliances',
    'Digital_Music'
]


/home/vic/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


NumPy version: 1.26.4
SciPy version: 1.10.1
Gensim version: 4.3.2


# Εργασία ΤΕΔΕ part 1
----

## load datasets from the urls to save them locally



#### get datasets with load_dataset and store them to drive as .csv

In [ ]:
from tabulate import tabulate
from datasets import load_dataset, get_dataset_config_names


review_datasets = {}
meta_datasets = {}
DATA_REPO = 'McAuley-Lab/Amazon-Reviews-2023'
for cat in categories:
  path_review = "raw_review_" + cat
  path_meta = "raw_meta_" + cat
  review_datasets[cat] = load_dataset(DATA_REPO,path_review,streaming=True,trust_remote_code=True)
  meta = {}
  meta = load_dataset(DATA_REPO,path_meta,streaming=True)
  meta_datasets[cat] = meta.remove_columns(['images','videos','author'])

for cat in categories:
  df_rev = pd.DataFrame(review_datasets[cat]['full'])
  #drop review entries that have these core columns empty and save them as files
  df_rev.dropna(subset=["rating","text","title"],inplace=True)
  #drop duplicates based on title
  df_rev.drop_duplicates(subset=['title'], keep='first', inplace=True)
  df_rev.to_csv(save_path + cat +"_reviews.csv", index=False)
  #any of the products with this column set to null are given the value false
  df_rev["verified_purchase"] = df_rev["verified_purchase"].fillna(False)


  df_meta = pd.DataFrame(meta_datasets[cat]['full'])
  #drop meta entries that have these core columns empty and save them as files
  df_meta = df_meta.dropna(subset=["title"])

  df_meta.to_csv(save_path + cat+"_meta.csv", index=False)





#### read to memory the .csv datasets
 



In [ ]:

#make a dictionary of the result of read_csv with keys as the names of the csv files
csv_list = []
for cat in categories:
  csv_list.append(read_csv(cat+"_reviews.csv"))
  csv_list.append(read_csv(cat+"_meta.csv"))

csv_keys = []
for cat in categories:
  csv_keys.append(cat+"_reviews.csv")
  csv_keys.append(cat+"_meta.csv")

csv_dict = {}
for key, value in zip(csv_keys, csv_list):
  csv_dict[key] = value


## Task1

###### Q1

In [ ]:
#print bars of each category's average next to the total average, part of Q1
def print_average_chart(categories,avgs=[]):
    num_categories = len(categories)
    category_averages = avgs[:-1]
    total_average = avgs[-1]
    x_indices = np.arange(num_categories)
    bar_width = 0.35 #each bar's width
    positions1 = x_indices-bar_width/2
    positions2 = x_indices+bar_width/2
    total_average_repeated = [total_average] * num_categories
    fig, ax = plt.subplots(figsize=(12, 7))
    rects1 = ax.bar(positions1, category_averages, bar_width, label='Category Average')
    rects2 = ax.bar(positions2, total_average_repeated, bar_width, label=f'Overall Average ({total_average:.2f})', color='#e5ae38')

    ax.set_title('Category Average (blue bars) vs. Overall Average Rating (yellow bars)', fontsize=16)
    ax.set_xticks(x_indices)
    ax.set_xticklabels(categories)

    #labels on top of bars
    ax.bar_label(rects1, padding=3, fmt='%.2f')
    ax.bar_label(rects2, padding=3, fmt='%.2f')

    plt.show()
    print("\n\n")


#print distribution of ratings, other part of Q1
def print_rating_distribution(ds,category_name):
  rating_counts = ds['rating'].value_counts().sort_index()

  #plot chart
  fig, ax = plt.subplots(figsize=(10, 6))
  bars = ax.bar(rating_counts.index, rating_counts.values, width=0.5)
  ax.set_title(f'Distribution of Product Ratings for Category: {category_name}', fontsize=16)
  ax.set_xlabel('Rating Values')
  ax.set_ylabel('Count')
  ax.bar_label(bars)

  plt.show()
  print("\n")


############################### Question 1 ########################################
print("-"*50 + " Question 1 " + "-"*50)
for cat in categories:
  print_rating_distribution(csv_dict[f"{cat}_reviews.csv"],cat)


print("\n\n")


#get average of each category and add them to a list
averages=[]
for cat in categories:
  averages.append(csv_dict[f"{cat}_meta.csv"]["average_rating"].mean())


#total average of all categories
total_avg=0.0
for avg in averages:
  total_avg+=avg
total_avg/=5
averages.append(total_avg)
print_average_chart(categories,averages)


#### Q2

In [ ]:
#for Q2
from tabulate import tabulate


# Helper: clean review text
def clean_review_text(text):
    text = html.unescape(text)  # Decode HTML entities
    text = re.sub(r'<[^>]+>', ' ', text)  # Remove HTML tags
    text = re.sub(r"n't", ' not', text)
    text = re.sub(r"'re", ' are', text)
    text = re.sub(r"'s", ' is', text)
    text = re.sub(r"'d", ' would', text)
    text = re.sub(r"'ll", ' will', text)
    text = re.sub(r"'ve", ' have', text)
    text = re.sub(r"'m", ' am', text)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)  # Remove non-letters
    return text.lower()

def common_phrases(ds,review_ds,name):
  print(f"For Category: {name}")
  #find the products with number of reviews but low ratings
  REVIEWS_THRESHOLD=50
  RATING_THRESHOLD=2.5
  filtered_ds=ds[(ds['rating_number'] >= REVIEWS_THRESHOLD) & (ds['average_rating'] <= RATING_THRESHOLD)]
  if(filtered_ds.empty):
    print(f"No products in category: {name} found with the given thresholds!")
    return

  results=[]
  #find for each of these products all reviews and track common words/phrases
  for _, row in filtered_ds.iterrows():
      # print(row)
      parent_asin = row['parent_asin']
      product_title = row.get('title')

      #get all reviews for this product that have a value in 'text' column \/
      product_reviews = review_ds[review_ds['parent_asin'] == parent_asin]['text'].dropna()

      if product_reviews.empty:
        print(f"No reviews found for ASIN: {parent_asin} and title: {product_title}")
        continue

      # clean reviews from HTML elements etc (found some br in the results)
      cleaned_reviews = product_reviews.apply(clean_review_text)

      #apply count vectorizer
      vectorizer = CountVectorizer(ngram_range=(1, 3), stop_words="english",max_features=15)
      X = vectorizer.fit_transform(cleaned_reviews)

      #count frequencies
      word_counts = X.toarray().sum(axis=0)
      keywords = pd.Series(word_counts, index=vectorizer.get_feature_names_out()).sort_values(ascending=False)


      #display results

      results.append({
            'ASIN': parent_asin,
            'Title': product_title,
            # 'Number of Reviews (review ds)': len(product_reviews), #gives back wrong results, maybe doesnt contain ALL reviews
            'Number of Reviews (meta ds)' : row['rating_number'],
            'Average Rating': row['average_rating'],
            'Keywords': keywords.to_dict()  # Convert keywords to a dictionary

      })
      # print(f"ASIN: {parent_asin} | Title: {product_title}")

      # print("Most common phrases in reviews:")
      # print(keywords)

  return (tabulate(results, headers="keys", tablefmt="psql"))


################################ Question 2 ########################################
print("-"*50 + " Question 2 " + "-"*50)
for cat in categories:
  print(common_phrases(csv_dict[f"{cat}_meta.csv"],csv_dict[f"{cat}_reviews.csv"],cat))
# common_phrases(beauty_ds, beauty_reviews)




#### Q3

In [ ]:
#for Q3, thelei ta TOP 5 apo oles tis katigories, oxi top 5 gia KATHE katigoria
def find_top5(ds=[]):
    # Get top 5 from each dataset
    top_lists = [df.nlargest(5, 'rating_number') for df in ds]

    # Concatenate all top 5s
    combined_top = pd.concat(top_lists, ignore_index=True)

    # Get the overall top 5
    top5 = combined_top.nlargest(5, 'rating_number')

    return top5[['title', 'features', 'rating_number']]

################################ Question 3 ########################################
print("\n\n" + "-"*50 + " Question 3 " + "-"*50 + "\n\n")
# find_top5([beauty_ds,software_ds,video_games_ds,electronics_ds,digital_music_ds])
find_top5([csv_dict[f"{cat}_meta.csv"] for cat in categories])


#### Q4

In [ ]:
#for Q4
def plot_ratings_over_time(df,name,start,end):
  # print(type(df))
  #convert timestamp to date time
  df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
  # we are interesting in plotting only average of each month between Jan. 1 2000 and Dec 31 2004
  start_date = pd.Timestamp(start)
  end_date = pd.Timestamp(end)
  filtered_df = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)].copy()
  #group by month
  filtered_df['month'] = filtered_df['datetime'].dt.to_period('M')
  monthly_grouped = filtered_df.groupby('month')
  monthly_means = monthly_grouped['rating'].mean() #average for each month
  # print(monthly_grouped.head(10))
  # print(monthly_medians)
  # Generate full month range between start and end
  full_month_range = pd.period_range(start=start_date, end=end_date, freq='M')

  # Reindex to include all months, even if they don't have data
  monthly_means = monthly_means.reindex(full_month_range)
  #FOR TESTING, to see how plow will be after some NaN values an actual entry to be added!
  # monthly_means[pd.Period('2024-12', freq='M')] = 3.2

  # print(monthly_means)

  #plot mean
  plt.figure(figsize=(20, 8))
  plt.plot(monthly_means.index.to_timestamp(), monthly_means.values, marker='o', linestyle='-')
  plt.title(f'Mean Rating Of {name} Over The Months')
  plt.xlabel('Month')
  plt.ylabel('Mean Rating')
  plt.grid(False)
  #add vertical dotted lines for readability
  for x in monthly_means.index.to_timestamp():
    plt.axvline(x=x, color='gray', linestyle=':', linewidth=0.5)

  #add the value of each dot above it for readability
  for x, y in zip(monthly_means.index.to_timestamp(), monthly_means.values):
    if not pd.isna(y):  # Only label non-NaN points
        plt.text(x, y + 0.005, f'{y:.2f}', ha='center', va='bottom', fontsize=8)


  # Format x-axis labels as mm/yy
  ax = plt.gca()
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%y'))

  # Set x-ticks to every month in the full range
  plt.xticks(monthly_means.index.to_timestamp(), rotation=45)
  plt.show()
  print("\n\n")


################################ Question 4 ########################################
print("\n\n" + "-"*50 + " Question 4 " + "-"*50 + "\n\n")

#give user freedom to pick date period, format is YYYY-MM-DD
start_date="2020-01-01"
end_date="2020-12-31"

for cat in categories:
  plot_ratings_over_time(csv_dict[f"{cat}_reviews.csv"],cat,start_date,end_date)


## Task 2

In [14]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm
import os
import time
import json
import gc

# Configuration
# device = 0 if torch.cuda.is_available() else -1
device = -1
MODEL_ID = "DT12the/distilbert-sentiment-analysis"
SAVE_PATH = "TEDE/"
CHECKPOINT_FILE = os.path.join(SAVE_PATH, "sentiment_checkpoint.json")
BATCH_SIZE = 16  # Increased for better GPU utilization
CHUNK_SIZE = 500  # Process 2000 reviews at a time
MAX_ROWS_PER_CATEGORY = 10000  # Max rows to process per category
CATEGORIES = [
    # 'Software',
    'Handmade_Products',
    # 'Appliances',
    # 'Digital_Music'
]

# Custom label mapping
DT12_LABELS = {
    0: 'POSITIVE',
    1: 'NEGATIVE'
}

# Initialize checkpoint system
def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            return json.load(f)
    return {
        "completed_categories": [],
        "current_category": "",
        "processed_chunks": 0,
        "processed_rows": 0,  # Track total processed rows per category
        "total_chunks": 0
    }

def save_checkpoint(state):
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump(state, f)

def delete_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        os.remove(CHECKPOINT_FILE)

# Processing functions
def normalize_rating(rating):
    return (rating - 1) / 4

def calculate_sentiment_score(sentiment_result, normalized_rating, w1=0.35, w2=0.65):
    label = sentiment_result['label'].upper()
    score = sentiment_result['score']

    if label == 'POSITIVE':
        return w1 * score + w2 * normalized_rating
    elif label == 'NEGATIVE':
        return w1 * (1 - score) + w2 * normalized_rating
    else:  # Shouldn't happen with our model
        return w1 * 0.5 + w2 * normalized_rating

# Initialize model pipeline (only once)
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID)

# Apply custom labels
model.config.id2label = DT12_LABELS
model.config.label2id = {v: k for k, v in DT12_LABELS.items()}

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=device,
    truncation=True,
    max_length=512
)

# # Load checkpoint
# checkpoint = load_checkpoint()
# print("Checkpoint loaded:", checkpoint)

# # Determine where to resume
# if checkpoint['current_category']:
#     print(f"Resuming from {checkpoint['current_category']} at chunk {checkpoint['processed_chunks']} "
#           f"with {checkpoint['processed_rows']} rows processed")
#     current_category = checkpoint['current_category']
#     categories_to_process = [c for c in CATEGORIES if c not in checkpoint['completed_categories']]

#     # Make sure we process the interrupted category first
#     if current_category in categories_to_process:
#         categories_to_process.remove(current_category)
#         categories_to_process.insert(0, current_category)
# else:
#     categories_to_process = [c for c in CATEGORIES if c not in checkpoint['completed_categories']]

# Process categories
categories = [
    # 'Software',
    # 'Handmade_Products',
    # 'Appliances',
    'Digital_Music'
]
for cat in categories:
    print(f"\n{'='*50}")
    print(f"Processing category: {cat}")
    print(f"{'='*50}")

    input_path = os.path.join(SAVE_PATH, f"{cat}_reviews.csv")
    output_path = os.path.join(SAVE_PATH, f"{cat}_sentiment_score.csv")

    # Create output file if not exists
    if not os.path.exists(output_path):
        pd.DataFrame(columns=['review_text', 'rating', 'normalized_rating',
                             'model_label', 'model_score', 'final_sentiment_score']
                    ).to_csv(output_path, index=False)

    # Get total rows in file for progress estimation
    if not os.path.exists(input_path):
        print(f"Input file not found: {input_path}")
        continue

    # Initialize row counter for this category
    # processed_rows = checkpoint.get('processed_rows', 0)
    processed_rows = 0

    # # Update checkpoint state
    # checkpoint.update({
    #     "current_category": cat,
    #     "processed_rows": processed_rows
    # })
    # save_checkpoint(checkpoint)

    # Create progress bar (row-based)
    pbar = tqdm(total=MAX_ROWS_PER_CATEGORY, initial=processed_rows,
                desc=f"Processing {cat}", unit='row')

    # Process chunks
    reader = pd.read_csv(input_path, chunksize=CHUNK_SIZE)

    for chunk_idx, chunk in enumerate(reader):
        try:
            chunk_start = time.time()

            # Check if we've reached row limit
            if processed_rows >= MAX_ROWS_PER_CATEGORY:
                print(f"\nReached {MAX_ROWS_PER_CATEGORY} rows for {cat}. Stopping processing.")
                break

            # Preprocess chunk - FIXED: Use .loc for assignment to avoid warnings
            # First create a clean copy of the chunk we're going to work with
            chunk = chunk.copy()

            # Now safely modify using .loc
            chunk.loc[:, 'text'] = chunk['text'].astype(str)
            chunk = chunk.dropna(subset=['text', 'rating'])

            if chunk.empty:
                print(f"Skipping empty chunk {chunk_idx}")
                pbar.update(0)
                continue

            # Calculate remaining capacity
            remaining = MAX_ROWS_PER_CATEGORY - processed_rows
            if len(chunk) > remaining:
                chunk = chunk.head(remaining)

            # Process sentiment in batches
            texts = chunk['text'].tolist()
            sentiment_results = sentiment_pipeline(texts, batch_size=BATCH_SIZE)

            # Calculate scores
            result_data = []
            for i, row in enumerate(chunk.itertuples()):
                norm_rating = normalize_rating(row.rating)
                result_data.append({
                    'review_text': row.text,
                    'rating': row.rating,
                    'normalized_rating': norm_rating,
                    'model_label': sentiment_results[i]['label'],
                    'model_score': sentiment_results[i]['score'],
                    'final_sentiment_score': calculate_sentiment_score(
                        sentiment_results[i],
                        norm_rating
                    )
                })

            # Append results to output file
            pd.DataFrame(result_data).to_csv(output_path, mode='a', header=False, index=False)

            # Update row counter
            processed_this_chunk = len(chunk)
            processed_rows += processed_this_chunk

            # Update checkpoint
            chunk_idx += 1

            # Update progress bar
            pbar.update(processed_this_chunk)

            # Print performance
            chunk_time = time.time() - chunk_start
            reviews_sec = processed_this_chunk / chunk_time
            pbar.set_postfix({
                "speed": f"{reviews_sec:.1f} rev/s",
                "last_chunk": f"{chunk_time:.1f}s",
                "total_rows": processed_rows
            })

        except Exception as e:
            print(f"\nError processing chunk {chunk_idx}: {str(e)}")
            print("Saving progress... You can resume later.")
            pbar.close()
            raise  # Re-raise exception to see full trace

        # Clean up memory
        del chunk, texts, sentiment_results, result_data
        gc.collect()
        # torch.cuda.empty_cache()
    pbar.close()
    print(f"\nCompleted {cat} with {processed_rows} rows processed")

Loading model...


Device set to use cpu



Processing category: Digital_Music


Processing Digital_Music: 100%|██████████| 10000/10000 [44:51<00:00,  3.71row/s, speed=3.9 rev/s, last_chunk=128.0s, total_rows=1e+4]


Reached 10000 rows for Digital_Music. Stopping processing.

Completed Digital_Music with 10000 rows processed


#### Weighted Combination of Text Sentiment and Rating

In [ ]:
import pandas as pd
from transformers import pipeline
import torch
from tqdm import tqdm

# Configure device
device = 0 if torch.cuda.is_available() else -1
# MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
MODEL = "DT12the/distilbert-sentiment-analysis"
save_path = "/content/drive/MyDrive/TEDE/"
# Initialize pipeline once (reuse across categories)
# Load model and tokenizer separately to customize labels
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)
DT12_LABELS = {
    0:'POSITIVE',
    1:'NEGATIVE'
}
# Apply custom labels
model.config.id2label = DT12_LABELS
model.config.label2id = {v: k for k, v in DT12_LABELS.items()}

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=device,
    truncation=True,
    max_length=512
)
# sentiment_pipeline = pipeline(
#     "sentiment-analysis",
#     model=MODEL,
#     device=device,
#     truncation=True,
#     max_length=512
# )

# Define processing functions
def normalize_rating(rating):
    return (rating - 1) / 4

def calculate_sentiment_score(sentiment_result, normalized_rating, w1=0.35, w2=0.65):
    label = sentiment_result['label'].upper()
    score = sentiment_result['score']

    if label == 'POSITIVE':
        return w1 * score + w2 * normalized_rating
    elif label == 'NEGATIVE':
        return w1 * (1 - score) + w2 * normalized_rating
    else:  # NEUTRAL
        return w1 * 0.5 + w2 * normalized_rating
categories=[
    # 'All_Beauty',
    'Software',
    'Handmade_Products',
    'Appliances',
    'Digital_Music'
]
# Process each category sequentially
for cat in categories:
    print(f"Processing {cat}...")

    # Configure paths
    input_path = f"{cat}_reviews.csv"
    output_path = f"{save_path}{cat}_sentiment_score.csv"

    # Create empty output file with headers
    pd.DataFrame(columns=['review_text', 'rating', 'normalized_rating',
                         'model_label', 'model_score', 'final_sentiment_score']
                ).to_csv(output_path, index=False)

    # Process in chunks
    chunk_size = 2000  # Adjust based on memory
    reader = pd.read_csv(os.path.join(save_path,input_path), chunksize=chunk_size)
    chunk_num = 0;
    for chunk in tqdm(reader, desc=f"Processing {cat}"):
        # Preprocess chunk
        chunk = chunk.dropna(subset=['text', 'rating'])
        chunk['text'] = chunk['text'].astype(str)

        if chunk.empty:
            continue

        # Process sentiment in batches
        texts = chunk['text'].tolist()
        sentiment_results = sentiment_pipeline(texts, batch_size=32)

        # Calculate scores
        # chunk['normalized_rating'] = chunk['rating'].apply(normalize_rating)
        result_data = []
        for i, row in enumerate(chunk.itertuples()):
            norm_rating = normalize_rating(row.rating)
            result_data.append({
                'review_text': row.text,
                'rating': row.rating,
                # 'normalized_rating': row['normalized_rating'],
                'normalized_rating': norm_rating,
                'model_label': sentiment_results[i]['label'],
                'model_score': sentiment_results[i]['score'],
                'final_sentiment_score': calculate_sentiment_score(
                    sentiment_results[i],
                    norm_rating
                )
            })

        # Append results to output file
        pd.DataFrame(result_data).to_csv(output_path, mode='a', header=False, index=False)
        chunk_num += 1
        if chunk_num == 100:
          break

    print(f"Completed {cat}")

In [ ]:
import torch


def get_normalized_rating(rating_list):
  return [(rating - 1) / 4 for rating in rating_list] #avoid .append() for better perfomance???

def get_final_sentiment_score(normalized_rating, text_sentiment, w1=0.35, w2=0.65):
    final_scores = []
    for i in range(len(text_sentiment)):
        label = text_sentiment[i]['label'].upper()
        score = text_sentiment[i]['score']
        if label == 'POSITIVE':
            sentiment_score = score
        elif label == 'NEGATIVE':
            sentiment_score = 1 - score
        else:  #NEUTRAL
            sentiment_score = 0.5
        final_score = w1 * sentiment_score + w2 * normalized_rating[i]
        final_scores.append(final_score)

    return final_scores


def create_csv(text,rating,normalized_rating,model_result,final_sentiment_score):
  out_ds=pd.DataFrame({
        'review_text': text,
        'rating': rating,
        'normalized_rating': normalized_rating,
        # 'model_score': [result['score'] for result in model_result], #maybe both label and score should be kept
        'model_score': model_result,
        'final_sentiment_score': final_sentiment_score
    })
  return out_ds



# Check if a GPU is available and set the device accordingly
# device=0 means the first GPU, device=-1 means the CPU
device = 0 if torch.cuda.is_available() else -1
#disconnected for gpu ffs
# device=-1
print(f"Using device: {'GPU' if device == 0 else 'CPU'}")

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
# tokenizer = AutoTokenizer.from_pretrained(MODEL)
# config = AutoConfig.from_pretrained(MODEL)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL)
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=MODEL,
    # tokenizer=tokenizer,
    device=device
)

# entries_num=50

for cat in categories:
  current_df = csv_dict[f"{cat}_reviews.csv"].copy()
  #to be aligned with task3 - part2, we drop any that dont have ratings or text
  current_df.dropna(subset=['text', 'rating'], inplace=True)
  current_df['text'] = current_df['text'].astype(str)
  text_list = current_df["text"].tolist()
  rating_list = current_df['rating'].tolist()

  #get result of model
  model_result= sentiment_pipeline(
      text_list,
      batch_size=32,
      truncation=True,
      max_length=512
  )
  # print(model_result)

  #normalise ratings from the reviews dataset
  normalized_rating=get_normalized_rating(rating_list)

  #calculate final score with weights
  final_sentiment_score=get_final_sentiment_score(normalized_rating,model_result)

  #create a new .csv file to save the columns
  out_ds =create_csv(text_list,rating_list,normalized_rating,model_result,final_sentiment_score)

  #save file
  out_ds.to_csv(f"{save_path+cat}_sentiment_score.csv", index=False)

  display(out_ds.tail(15))


# Εργασία ΤΕΔΕ part 2
--------

## Task 1

In [ ]:


def find_best_k_by_elbow(k_values, inertia_values):
    knee = KneeLocator(
        k_values,
        inertia_values,
        curve='convex',
        direction='decreasing'
    )
    return knee.knee #returns optimal k.



for cat in (categories):

  #we retrieve all columns but we later only keep the ones that matter
  whole_ds=csv_dict[cat+"_meta.csv"]
  #keep only  description, rating, price
  ds=whole_ds[['description','average_rating','price']].copy()

  # --- Check nulls in the columns of interest BEFORE selection and dropna ---
  print("\nNull counts in raw data for relevant columns:")
  print(ds.isnull().sum())

  #drop any entry that has null in description or average_rating
  ds = ds.dropna(subset=['description','average_rating','price']).copy()


  #REPLACE PRICE WITH MEAN instead of dropping the null values
  # mean_price = ds['price'].mean()
  # ds.loc[:, 'price'] = ds['price'].fillna(mean_price)

  print(f"Length before description processing: {len(ds)}")

  #clean description
  ds.loc[:, 'description'] = ds['description'].apply(preprocess_text)
  #drop also those that have nothing left after processing
  ds = ds[ds['description'].str.strip().astype(bool)].copy()
  print(f"Length after description processing: {len(ds)}")


  #initialize scaler and apply it
  scaler = StandardScaler()
  #apply scaler on both numerical columns
  scaled_numerical_features = scaler.fit_transform(ds[['price', 'average_rating']])
  # scaled_numerical_features = scaler.fit_transform(ds[['average_rating']])


  #try some max_features values
  mf_range=[1000,2000,5000,7000,10000,15000,20000]
  # mf_range=[200]
  for mf in mf_range:
    print(f"for mf={mf}")
    #apply tf-idf to description column to get scaled results for words
    tfidf_vectorizer = TfidfVectorizer(
        max_features=mf,
        stop_words=None,  #no need, we already preprocess it in function
        norm='l2',
        min_df=5, #must appear on at least n documents
        max_df=0.5, #discard anything that appears over the specified percentage in all documents - avoid other common words
        ngram_range=(1, 2) #try some n-grams
    )
    tfidf_features = tfidf_vectorizer.fit_transform(ds['description'])

    #combine scaled price and rating with tf-idf features with hstack for efficiency
    combined_features = hstack((tfidf_features, scaled_numerical_features))


    # print(combined_features)

    #implement clustering with k-means algorithm
    k_range = range(2, 11)
    inertia_values = [] # List to store inertia for each k

    # print("calculating inertia for different values of K...")
    for k in k_range:
        # initialize KMeans for the current K, random_state for reproducibility
        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        kmeans.fit(combined_features)
        #append the inertia (WCSS) to our list
        inertia_values.append(kmeans.inertia_)
        # print(f"Inertia for K={k}: {kmeans.inertia_:.2f}")

    # best_k = find_best_k_by_elbow(k_range, inertia_values)
    # print(f"The optimal number of clusters is: k = {best_k}")


    #plot the Elbow Curve to determine optimal k, already done and recorded
    plt.figure(figsize=(10, 6))
    plt.plot(k_range, inertia_values, marker='o',color="purple", linestyle='-')
    plt.title('Elbow Method for Optimal K')
    plt.xlabel('Number of Clusters (K)')
    plt.ylabel('Inertia (WCSS - Within-Cluster Sum of Squares)')
    plt.xticks(k_range) # Ensure all tested K values are shown on the x-axis
    plt.grid(True)
    plt.show()

    #calculate silhouette score
    for k in k_range:
        final_kmeans = KMeans(n_clusters=k, random_state=42, n_init="auto")
        cluster_labels = final_kmeans.fit_predict(combined_features)
        ds['cluster'] = cluster_labels
        silhouette_avg = silhouette_score(combined_features, cluster_labels)
        print(f"For n_clusters = {k}, the average silhouette_score is: {silhouette_avg:.4f}")


    #PCA visualization
    #instanciate pca, set visualization to 2 dimensions
    # pca = PCA(n_components=2, random_state=42)

    # #.toarray() converts the sparse matrix from hstack into a standard dense numpy array.
    # features_2d = pca.fit_transform(combined_features.toarray())

    # df_plot = pd.DataFrame({
    #     'pca_1': features_2d[:, 0],
    #     'pca_2': features_2d[:, 1],
    #     'cluster': cluster_labels
    # })

    # plt.figure(figsize=(12, 8))

    # #create plots using seaborn
    # sns.scatterplot(
    #     data=df_plot, #the dataFrame to use for plotting
    #     x='pca_1',  #set the x-axis to our first principal component
    #     y='pca_2', #set the y-axis to our second principal component
    #     hue='cluster', #colour each point based on the cluster it belongs
    #     palette='viridis', #colour scheme
    #     s=50, #size of points
    #     alpha=0.7 #transparency
    # )
    # plt.title(f'Product Clusters for {cat} (Visualized with PCA)', fontsize=16)
    # plt.xlabel('Principal Component 1', fontsize=12)
    # plt.ylabel('Principal Component 2', fontsize=12)
    # plt.legend(title='Cluster')
    # plt.grid(True)
    # plt.show()




## Task 2

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
from scipy.sparse import csr_matrix

# --- Configuration ---
CATEGORY = 'Software'
SUBSET_ROWS = 100000  # Using a 20k row subset as requested
USER_ID_COL = 'user_id'
ITEM_ID_COL = 'parent_asin'
RATING_COL = 'rating'
SAVE_PATH = "TEDE/"
TOP_K = 10 # Number of recommendations to generate

def create_sparse_matrix(df, user_col, item_col, rating_col):
    # Create mappings
    users = df[user_col].unique()
    items = df[item_col].unique()
    user_to_idx = {user: idx for idx, user in enumerate(users)}
    item_to_idx = {item: idx for idx, item in enumerate(items)}
    
    # Create sparse matrix
    rows = df[user_col].map(user_to_idx)
    cols = df[item_col].map(item_to_idx)
    data = df[rating_col].values
    
    sparse_matrix = csr_matrix((data, (rows, cols)), shape=(len(users), len(items)))
    
    return sparse_matrix, user_to_idx, item_to_idx, users, items

# --- Load Data ---
print(f"Loading data for category: {CATEGORY}...")
print(f"Using a subset of {SUBSET_ROWS} reviews.")
try:
    # Load a subset of reviews for user-item interaction data
    reviews_df = pd.read_csv(f"{SAVE_PATH}{CATEGORY}_reviews.csv", low_memory=True, nrows=SUBSET_ROWS)
    # Load metadata for content-based features
except FileNotFoundError:
    print(f"Error: Make sure '{CATEGORY}_reviews.csv' and '{CATEGORY}_meta.csv' exist in '{SAVE_PATH}'")
    # Exit or handle error appropriately
    exit()

# --- Prepare Data for Collaborative Filtering (CF) ---
print("Preparing data for Collaborative Filtering...")
# Select necessary columns and drop duplicates/missing values
cf_df = reviews_df[[USER_ID_COL, ITEM_ID_COL, RATING_COL]].copy()
cf_df.dropna(inplace=True)

# To reduce computation time, let's filter for users and items with a minimum number of interactions
min_user_reviews = 5
min_item_reviews = 5


user_counts = cf_df[USER_ID_COL].value_counts()
item_counts = cf_df[ITEM_ID_COL].value_counts()

cf_df = cf_df[cf_df[USER_ID_COL].isin(user_counts[user_counts >= min_user_reviews].index)]
cf_df = cf_df[cf_df[ITEM_ID_COL].isin(item_counts[item_counts >= min_item_reviews].index)]

print(f"Filtered CF data to {len(cf_df)} interactions.")

# Split data into training and testing sets
train_df, test_df = train_test_split(cf_df, test_size=0.2, random_state=42)
print(f"Training set size: {len(train_df)}, Test set size: {len(test_df)}")

# Create a user-item matrix for training data
user_item_matrix = train_df.pivot_table(index=USER_ID_COL, columns=ITEM_ID_COL, values=RATING_COL).fillna(0)
print("\nUser-Item matrix created.")
# FREE MEMORY - Delete large DataFrames no longer needed
del reviews_df, cf_df
import gc
gc.collect()
# display(user_item_matrix.shape[0], " users and " + str(user_item_matrix.shape[1]) + " items.")

Loading data for category: Software...
Using a subset of 100000 reviews.
Preparing data for Collaborative Filtering...
Filtered CF data to 24398 interactions.
Training set size: 19518, Test set size: 4880

User-Item matrix created.


0

CF

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import seaborn as sns
from scipy.sparse import hstack
# --- Collaborative Filtering Implementation ---

# Calculate similarity matrices
# User-User similarity
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
# Use float32 for similarity matrices to halve memory usage
user_similarity = cosine_similarity(user_item_matrix.values.astype(np.float32))
user_similarity_df = pd.DataFrame(
    user_similarity.astype(np.float32),  # Store as float32
    index=user_item_matrix.index, 
    columns=user_item_matrix.index
)

# Free intermediate arrays
del user_similarity
gc.collect()
# Same for item similarity
item_similarity = cosine_similarity(user_item_matrix.T.values.astype(np.float32))
item_similarity_df = pd.DataFrame(
    item_similarity.astype(np.float32),
    index=user_item_matrix.columns, 
    columns=user_item_matrix.columns
)

del item_similarity
gc.collect()

print("User-User and Item-Item similarity matrices calculated.")

def predict_rating(user_id, item_id, user_item_matrix, similarity_df, k=15, is_user_based=True):
    """Predicts a rating for a user-item pair."""
    if is_user_based:
        if user_id not in similarity_df.index: return 0
        user_mean = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].mean()
        # Get top k similar users who have rated the item
        sim_users = similarity_df[user_id].sort_values(ascending=False)[1:k+1]
        rated_users = user_item_matrix.loc[:, item_id][user_item_matrix.loc[:, item_id] > 0].index
        top_k_users = sim_users[sim_users.index.isin(rated_users)]
        
        if top_k_users.empty: return user_mean  # Return user's average instead of 0
        
        # Mean-centered weighted average
        weighted_sum = 0
        sum_of_weights = 0
        
        for similar_user in top_k_users.index:
            similar_user_mean = user_item_matrix.loc[similar_user][user_item_matrix.loc[similar_user] > 0].mean()
            rating_deviation = user_item_matrix.loc[similar_user, item_id] - similar_user_mean
            weight = top_k_users[similar_user]
            
            weighted_sum += weight * rating_deviation
            sum_of_weights += abs(weight)
        
        if sum_of_weights == 0:
            return user_mean
        
        prediction = user_mean + (weighted_sum / sum_of_weights)
        # Clamp prediction to valid rating range
        return max(1.0, min(5.0, prediction))
    else: # Item-based
        if item_id not in similarity_df.index: return 0
         # Get item's mean rating
        item_mean = user_item_matrix.loc[:, item_id][user_item_matrix.loc[:, item_id] > 0].mean()
        
        # Get top k similar items that the user has rated
        sim_items = similarity_df[item_id].sort_values(ascending=False)[1:k+1]
        rated_items = user_item_matrix.loc[user_id, :][user_item_matrix.loc[user_id, :] > 0].index
        top_k_items = sim_items[sim_items.index.isin(rated_items)]

        if top_k_items.empty: return item_mean

        # Mean-centered weighted average
        weighted_sum = 0
        sum_of_weights = 0
        
        for similar_item in top_k_items.index:
            similar_item_mean = user_item_matrix.loc[:, similar_item][user_item_matrix.loc[:, similar_item] > 0].mean()
            rating_deviation = user_item_matrix.loc[user_id, similar_item] - similar_item_mean
            weight = top_k_items[similar_item]
            
            weighted_sum += weight * rating_deviation
            sum_of_weights += abs(weight)
        
        if sum_of_weights == 0:
            return item_mean
            
        prediction = item_mean + (weighted_sum / sum_of_weights)
        return max(1.0, min(5.0, prediction))

def recommend_cf(user_id, user_item_matrix, similarity_df, k=100, top_n=10, is_user_based=True):
    """Recommends top_n items for a user."""
    # Get items the user has not rated
    rated_items = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index
    all_items = user_item_matrix.columns
    items_to_predict = np.setdiff1d(all_items, rated_items)
    
    predictions = {}
    for item_id in items_to_predict:
        pred_rating = predict_rating(user_id, item_id, user_item_matrix, similarity_df, k, is_user_based)
        if pred_rating > 0:
            predictions[item_id] = pred_rating
            
    # Sort predictions and return top N
    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    return sorted_predictions[:top_n]

# --- Example Usage ---
# Pick a sample user from the training data
if not user_item_matrix.empty:
    sample_user_id = user_item_matrix.index[0] 
    print(f"\n--- Recommendations for User: {sample_user_id} ---")

    # User-Based CF Recommendations
    user_based_recs = recommend_cf(sample_user_id, user_item_matrix, user_similarity_df, is_user_based=True, top_n=TOP_K)
    print(f"\nTop {TOP_K} User-Based Recommendations:")
    print(pd.DataFrame(user_based_recs, columns=['Item ID', 'Predicted Rating']))

    # Item-Based CF Recommendations
    item_based_recs = recommend_cf(sample_user_id, user_item_matrix, item_similarity_df, is_user_based=False, top_n=TOP_K)
    print(f"\nTop {TOP_K} Item-Based Recommendations:")
    print(pd.DataFrame(item_based_recs, columns=['Item ID', 'Predicted Rating']))
else:
    print("\nCould not generate recommendations. The user-item matrix is empty after filtering.")
    print("Try increasing SUBSET_ROWS or lowering min_user_reviews/min_item_reviews.")

User-User and Item-Item similarity matrices calculated.

--- Recommendations for User: AE22MRZKBD5MWJLIMGYLLJ67ZWVQ ---

Top 10 User-Based Recommendations:
      Item ID  Predicted Rating
0  B004HE5TAG          5.000000
1  B00NG6CIKA          5.000000
2  B07CNPYVKS          4.803715
3  B00MJB8GRC          4.800000
4  B00ICQFSV0          4.633333
5  B076B488LJ          4.618182
6  B00M1EAQHK          4.481818
7  B00OBCO752          4.481818
8  B014WAAFOY          4.481818
9  B01C0JXY6U          4.481818

Top 10 Item-Based Recommendations:
      Item ID  Predicted Rating
0  B0000AZJV8               5.0
1  B0000E6NK9               5.0
2  B0002G71T0               5.0
3  B000B6N2O4               5.0
4  B000BAWLS8               5.0
5  B000BX5JQG               5.0
6  B000WMUHEU               5.0
7  B001AFFYSW               5.0
8  B001AMLRU4               5.0
9  B001E0RZ3U               5.0


CBF

In [11]:
# Replace your CBF preparation section with this simpler, correct version

# --- Prepare Data for Content-Based Filtering (CBF) ---
meta_df = pd.read_csv(f"{SAVE_PATH}{CATEGORY}_meta.csv", low_memory=True, nrows=SUBSET_ROWS)
print("\nPreparing data for Content-Based Filtering...")

# Use the metadata, focusing on text descriptions and price
cbf_df = meta_df[['parent_asin', 'title', 'description', 'price']].copy()
cbf_df.dropna(subset=['title', 'description'], inplace=True)
cbf_df.drop_duplicates(subset=['parent_asin'], inplace=True)

# Keep only items that are present in our CF dataset (either train OR test)
# This ensures consistency between CF and CBF recommendations
all_cf_items = set(train_df[ITEM_ID_COL].unique()).union(set(test_df[ITEM_ID_COL].unique()))
cbf_df = cbf_df[cbf_df[ITEM_ID_COL].isin(all_cf_items)]

# Handle missing prices by filling with median (more robust than mean for prices)
median_price = cbf_df['price'].median()
cbf_df['price'] = cbf_df['price'].fillna(median_price)

# Normalize price using Min-Max scaling (0-1 range)
from sklearn.preprocessing import MinMaxScaler
price_scaler = MinMaxScaler()
cbf_df['normalized_price'] = price_scaler.fit_transform(cbf_df[['price']])

# Combine title and description for a richer content source
cbf_df['content'] = cbf_df['title'].fillna('') + ' ' + cbf_df['description'].fillna('')

# Preprocess the text content using the function defined in previous tasks
cbf_df['processed_content'] = cbf_df['content'].apply(preprocess_text)

print(f"Content-based data prepared for {len(cbf_df)} items.")

# Display price distribution after normalization
print(f"Price normalization complete:")
print(f"Original price range: ${cbf_df['price'].min():.2f} - ${cbf_df['price'].max():.2f}")
print(f"Normalized price range: {cbf_df['normalized_price'].min():.3f} - {cbf_df['normalized_price'].max():.3f}")


Preparing data for Content-Based Filtering...
Content-based data prepared for 3027 items.
Price normalization complete:
Original price range: $0.00 - $599.99
Normalized price range: 0.000 - 1.000


In [12]:
embedding_model = api.load('fasttext-wiki-news-subwords-300')
embedding_dim = embedding_model.vector_size

In [13]:

# --- Content-Based Filtering Implementation ---

# Load pre-trained model (ensure you have an internet connection)
# This might take a few minutes to download on the first run
# try:
#     print("\nLoading pre-trained embedding model (fasttext-wiki-news-subwords-300)...")
#     embedding_model = api.load('fasttext-wiki-news-subwords-300')
#     embedding_dim = embedding_model.vector_size
#     print("Model loaded successfully.")
# except Exception as e:
#     print(f"Failed to load embedding model: {e}")
#     exit()

# Re-use the document_to_vector function from the previous task
def document_to_vector(text, model, num_features):
    words = text.split()
    feature_vec = np.zeros((num_features,), dtype="float32")
    n_words = 0
    for word in words:
        if word in model:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if n_words > 0:
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

# Create document vectors for all items
print("Creating item vectors from content...")
item_vectors = np.array([document_to_vector(doc, embedding_model, embedding_dim) for doc in cbf_df['processed_content']])
cbf_item_similarity = cosine_similarity(item_vectors)
cbf_item_similarity_df = pd.DataFrame(cbf_item_similarity, index=cbf_df[ITEM_ID_COL], columns=cbf_df[ITEM_ID_COL])
print("Content-based item similarity matrix created.")

def recommend_cbf(item_id, similarity_df, top_n=10):
    """Recommends items based on content similarity to a given item."""
    if item_id not in similarity_df.index:
        return []
    # Get similarity scores for the item, drop the item itself, and sort
    sim_scores = similarity_df[item_id].drop(item_id).sort_values(ascending=False)
    return list(zip(sim_scores.head(top_n).index, sim_scores.head(top_n).values))

# --- Example Usage ---
# Pick a sample item that the user has rated highly
if not user_item_matrix.empty:
    user_history = train_df[train_df[USER_ID_COL] == sample_user_id].sort_values(RATING_COL, ascending=False)
    if not user_history.empty:
        sample_item_id = user_history[ITEM_ID_COL].iloc[0]
        print(f"\n--- Recommendations based on item: {sample_item_id} ---")
        
        cbf_recs = recommend_cbf(sample_item_id, cbf_item_similarity_df, top_n=TOP_K)
        print(f"\nTop {TOP_K} Content-Based Recommendations (similar to {sample_item_id}):")
        print(pd.DataFrame(cbf_recs, columns=['Item ID', 'Similarity Score']))
    else:
        print(f"Sample user {sample_user_id} has no rating history in the training set.")

Creating item vectors from content...
Content-based item similarity matrix created.

--- Recommendations based on item: B010KKC7LO ---

Top 10 Content-Based Recommendations (similar to B010KKC7LO):
      Item ID  Similarity Score
0  B005YFNC92          0.951250
1  B00IKMBM3E          0.950435
2  B007GE0X3M          0.947009
3  B0756R3WNM          0.944436
4  B07RBQX2ZP          0.940458
5  B0055DL1G4          0.939161
6  B006L4S3OY          0.934329
7  B004SYNIJ4          0.933540
8  B004X7UPL0          0.933303
9  B006DRVFRQ          0.933123


Hybrid

In [23]:
# --- Hybrid Recommendation System ---

def recommend_hybrid(user_id, cf_recs, cbf_item_similarity_df, user_history_df, cf_weight=0.2, cbf_weight=0.8, top_n=10):
    """Combines CF and CBF recommendations."""
    hybrid_scores = {}
    
    # Get user's highly rated items for CBF context
    positive_history = user_history_df[user_history_df[RATING_COL] >= 4][ITEM_ID_COL]
    
    # Normalize CF scores (0 to 1)
    cf_scores = {item: score/5.0 for item, score in cf_recs}

    # Calculate CBF score for each CF recommendation
    for item_id, cf_score in cf_scores.items():
        cbf_score = 0
        if not positive_history.empty and item_id in cbf_item_similarity_df.index:
            # Avg similarity to user's liked items
            # Ensure all items in positive_history exist in the similarity matrix index
            valid_history = [item for item in positive_history if item in cbf_item_similarity_df.index]
            if valid_history:
                sims = cbf_item_similarity_df.loc[valid_history, item_id]
                cbf_score = sims.mean() if not sims.empty else 0
        
        # Combine scores
        hybrid_scores[item_id] = (cf_score * cf_weight) + (cbf_score * cbf_weight)
        
    sorted_hybrid = sorted(hybrid_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_hybrid[:top_n]

# --- Example Usage ---
if not user_item_matrix.empty:
    print(f"\n--- Hybrid Recommendations for User: {sample_user_id} ---")
    # Use item-based CF as the base
    item_based_recs_full = recommend_cf(sample_user_id, user_item_matrix, item_similarity_df, is_user_based=False, top_n=50)

    hybrid_recs = recommend_hybrid(sample_user_id, item_based_recs_full, cbf_item_similarity_df, user_history, top_n=TOP_K)
    print(f"\nTop {TOP_K} Hybrid Recommendations:")
    print(pd.DataFrame(hybrid_recs, columns=['Item ID', 'Hybrid Score']))


--- Hybrid Recommendations for User: AE22MRZKBD5MWJLIMGYLLJ67ZWVQ ---

Top 100 Hybrid Recommendations:
       Item ID  Hybrid Score
0   B003XKRZES      0.925339
1   B000BX5JQG      0.923467
2   B004M39SN6      0.923442
3   B003ZK51WY      0.918965
4   B004R1D7A8      0.918768
5   B002Q2TTIO      0.918094
6   B0050QV0RQ      0.917964
7   B004ULSHJ6      0.916612
8   B0043T8K8I      0.916132
9   B001AFFYSW      0.915725
10  B002DHLVII      0.914861
11  B001E0RZ3U      0.914572
12  B001AMLRU4      0.913373
13  B0055ELS7U      0.912335
14  B000WMUHEU      0.912160
15  B004U5XBZC      0.911922
16  B0043SK9QU      0.911747
17  B000BAWLS8      0.911231
18  B004XE06OY      0.909645
19  B004V4XOIQ      0.909479
20  B004TMPJP6      0.908979
21  B001W2Y5Q0      0.908344
22  B002JVX5V4      0.908259
23  B004W4IMUK      0.908124
24  B004SVMNXY      0.907784
25  B0030EZZA8      0.907687
26  B004ECJI9O      0.907288
27  B0002G71T0      0.907258
28  B004XIUDJ8      0.904870
29  B0055746JO      0.9048

--- Recommendations for User: AE22MRZKBD5MWJLIMGYLLJ67ZWVQ ---

Top 10 User-Based Recommendations:
      Item ID  Predicted Rating
0  B004HE5TAG          5.000000
1  B00NG6CIKA          5.000000
2  B07CNPYVKS          4.803715
3  B00MJB8GRC          4.800000
4  B00ICQFSV0          4.633333
5  B076B488LJ          4.618182
6  B00M1EAQHK          4.481818
7  B00OBCO752          4.481818
8  B014WAAFOY          4.481818
9  B01C0JXY6U          4.481818

Top 10 Item-Based Recommendations:
      Item ID  Predicted Rating
0  B0000AZJV8               5.0
1  B0000E6NK9               5.0
2  B0002G71T0               5.0
3  B000B6N2O4               5.0
4  B000BAWLS8               5.0
5  B000BX5JQG               5.0
6  B000WMUHEU               5.0
7  B001AFFYSW               5.0
8  B001AMLRU4               5.0
9  B001E0RZ3U               5.0

In [24]:
# Replace the evaluation section with this faster version

# --- Faster Evaluation with Sampling ---
print("\n" + "="*60)
print(f"--- Evaluating Ranking Performance with Recall@{TOP_K} (SAMPLED) ---")
print("="*60)

# Sample a smaller subset of test users for faster evaluation
# Test with more lenient criteria
SAMPLE_SIZE = 50  # Smaller sample for faster testing
relevant_threshold = 3.5  # Lower threshold
TOP_K = 100  # More recommendations
print(f"Sampling {SAMPLE_SIZE} users from {len(test_df.groupby(USER_ID_COL))} total test users for faster evaluation...")

def evaluate_recall_at_k_sampled(test_df, train_df, k, model_type, sample_size=200):
    """Same as before but with user sampling for speed"""
    total_recall = 0
    user_count = 0
    
    # Sample users instead of evaluating all
    test_user_groups = test_df.groupby(USER_ID_COL)
    sampled_users = list(test_user_groups.groups.keys())[:sample_size]  # Take first N users
    
    for user_id in tqdm(sampled_users, desc=f"Evaluating {model_type} (Recall@{k})"):
        if user_id not in user_item_matrix.index:
            continue
        
        group = test_user_groups.get_group(user_id)
        relevant_items = set(group[group[RATING_COL] >= relevant_threshold][ITEM_ID_COL])
        if not relevant_items:
            continue

        # Generate recommendations (same as before)
        recs = []
        if model_type == 'User-Based CF':
            recs = recommend_cf(user_id, user_item_matrix, user_similarity_df, is_user_based=True, top_n=k)
        elif model_type == 'Item-Based CF':
            recs = recommend_cf(user_id, user_item_matrix, item_similarity_df, is_user_based=False, top_n=k)
        elif model_type == 'Content-Based':
            user_history = train_df[train_df[USER_ID_COL] == user_id].sort_values(RATING_COL, ascending=False)
            if not user_history.empty:
                seed_item = user_history.iloc[0][ITEM_ID_COL]
                recs = recommend_cbf(seed_item, cbf_item_similarity_df, top_n=k)
        elif model_type == 'Hybrid':
            user_history = train_df[train_df[USER_ID_COL] == user_id]
            cf_recs_full = recommend_cf(user_id, user_item_matrix, item_similarity_df, is_user_based=False, top_n=50)
            recs = recommend_hybrid(user_id, cf_recs_full, cbf_item_similarity_df, user_history, top_n=k)
        
        if not recs:
            continue
            
        recommended_items = {item_id for item_id, score in recs}
        hits = len(relevant_items.intersection(recommended_items))
        recall_for_user = hits / len(relevant_items)
        total_recall += recall_for_user
        user_count += 1

    return total_recall / user_count if user_count > 0 else 0.0

# Run faster evaluation
results = {}
if not test_df.empty:
    # results['User-Based CF'] = evaluate_recall_at_k_sampled(test_df, train_df, k=TOP_K, model_type='User-Based CF', sample_size=SAMPLE_SIZE)
    # results['Item-Based CF'] = evaluate_recall_at_k_sampled(test_df, train_df, k=TOP_K, model_type='Item-Based CF', sample_size=SAMPLE_SIZE)
    # results['Content-Based'] = evaluate_recall_at_k_sampled(test_df, train_df, k=TOP_K, model_type='Content-Based', sample_size=SAMPLE_SIZE)
    results['Hybrid'] = evaluate_recall_at_k_sampled(test_df, train_df, k=TOP_K, model_type='Hybrid', sample_size=SAMPLE_SIZE)

    print("\n--- Final Model Comparison (Sampled Evaluation) ---")
    results_df = pd.DataFrame(list(results.items()), columns=['Model', f'Recall@{TOP_K}'])
    results_df = results_df.sort_values(by=f'Recall@{TOP_K}', ascending=False)
    print(results_df.to_string(index=False))
# from tqdm import tqdm
# import numpy as np
# import pandas as pd

# # --- Evaluation Function (Recall@K) ---
# def evaluate_recall_at_k(test_df, train_df, k, model_type):
#     """
#     Calculates Recall@K for all recommendation models.
#     This metric measures the percentage of relevant items (highly rated by the user in the test set)
#     that are successfully found in the top-K recommendations. Higher is better.
#     """
#     total_recall = 0
#     user_count = 0
    
#     # Group test data by user to evaluate each user individually
#     test_user_groups = test_df.groupby(USER_ID_COL)
    
#     for user_id, group in tqdm(test_user_groups, desc=f"Evaluating {model_type} (Recall@{k})"):
#         # Ensure the user exists in our training data matrix
#         if user_id not in user_item_matrix.index:  # FIXED: Use 'in' instead of .contains()
#             continue
        
#         # Ground Truth: These are the items the user actually liked in the test set.
#         # We define "liked" as a rating of 4.0 or higher.
#         relevant_items = set(group[group[RATING_COL] >= 4.0][ITEM_ID_COL])
#         if not relevant_items:
#             continue

#         # Generate top-K recommendations for the current user based on the model type
#         recs = []
#         if model_type == 'User-Based CF':
#             recs = recommend_cf(user_id, user_item_matrix, user_similarity_df, is_user_based=True, top_n=k)
#         elif model_type == 'Item-Based CF':
#             recs = recommend_cf(user_id, user_item_matrix, item_similarity_df, is_user_based=False, top_n=k)
#         elif model_type == 'Content-Based':
#             # For CBF, we need a "seed" item from the user's training history to find similar items
#             user_history = train_df[train_df[USER_ID_COL] == user_id].sort_values(RATING_COL, ascending=False)
#             if not user_history.empty:
#                 seed_item = user_history.iloc[0][ITEM_ID_COL]
#                 recs = recommend_cbf(seed_item, cbf_item_similarity_df, top_n=k)
#         elif model_type == 'Hybrid':
#             user_history = train_df[train_df[USER_ID_COL] == user_id]
#             # The hybrid model uses item-based CF as a starting point
#             cf_recs_full = recommend_cf(user_id, user_item_matrix, item_similarity_df, is_user_based=False, top_n=50)
#             recs = recommend_hybrid(user_id, cf_recs_full, cbf_item_similarity_df, user_history, top_n=k)
        
#         if not recs:
#             continue
            
#         # Extract just the item IDs from the recommendations
#         recommended_items = {item_id for item_id, score in recs}
        
#         # Calculate how many of the relevant items were in the recommended list
#         hits = len(relevant_items.intersection(recommended_items))
#         recall_for_user = hits / len(relevant_items)
#         total_recall += recall_for_user
#         user_count += 1

#     # Return the average recall across all users
#     return total_recall / user_count if user_count > 0 else 0.0

# # --- Run All Evaluations ---
# print("\n" + "="*60)
# print(f"--- Evaluating Ranking Performance with Recall@{TOP_K} ---")
# print("="*60)

# results = {}
# if not test_df.empty:
#     # Evaluate each model
#     results['User-Based CF'] = evaluate_recall_at_k(test_df, train_df, k=TOP_K, model_type='User-Based CF')
#     results['Item-Based CF'] = evaluate_recall_at_k(test_df, train_df, k=TOP_K, model_type='Item-Based CF')
#     results['Content-Based'] = evaluate_recall_at_k(test_df, train_df, k=TOP_K, model_type='Content-Based')
#     results['Hybrid'] = evaluate_recall_at_k(test_df, train_df, k=TOP_K, model_type='Hybrid')

#     # --- Display Final Results ---
#     print("\n--- Final Model Comparison ---")
#     results_df = pd.DataFrame(list(results.items()), columns=['Model', f'Recall@{TOP_K}'])
#     results_df = results_df.sort_values(by=f'Recall@{TOP_K}', ascending=False)
#     print(results_df.to_string(index=False))


--- Evaluating Ranking Performance with Recall@100 (SAMPLED) ---
Sampling 50 users from 2536 total test users for faster evaluation...


Evaluating Hybrid (Recall@100): 100%|██████████| 50/50 [01:56<00:00,  2.33s/it]


--- Final Model Comparison (Sampled Evaluation) ---
 Model  Recall@100
Hybrid    0.028571


### Analysis of Recommendation Approaches

#### 1. Collaborative Filtering (CF)
*   **Strengths**:
    *   **Serendipity**: Can recommend items that are not obviously similar, leading to novel discoveries for the user (e.g., users who bought X also bought Y, even if X and Y are in different genres).
    *   **No Domain Knowledge Needed**: Doesn't require any information about the items themselves, only user interaction data.
*   **Weaknesses**:
    *   **Cold Start Problem**: It cannot make recommendations for new users (no rating history) or new items (no ratings received). Our implementation handles this by returning no recommendations.
    *   **Data Sparsity**: Performance suffers when the user-item matrix is sparse (most users have rated very few items), making it hard to find overlapping users or items.
    *   **Popularity Bias**: Tends to recommend popular items more frequently, as they have more ratings.

#### 2. Content-Based Filtering (CBF)
*   **Strengths**:
    *   **Solves Item Cold Start**: Can recommend new items as soon as they are added, provided they have descriptive content.
    *   **User Independence**: Recommendations for one user do not depend on the ratings of other users.
    *   **Explainability**: Recommendations are easy to explain (e.g., "Recommended because it is similar to X, which you liked").
*   **Weaknesses**:
    *   **Limited Serendipity**: Tends to recommend items that are very similar to what a user has already liked, leading to a "filter bubble" and a lack of diversity.
    *   **Requires Feature Engineering**: The quality of recommendations is highly dependent on the quality of the item content (descriptions, tags) and the feature extraction process.

#### 3. Hybrid Approach
*   **Strengths**:
    *   **Mitigates Weaknesses**: By combining CF and CBF, it can alleviate the major problems of each. For instance, it can reduce the cold start problem and data sparsity issues of CF while introducing more serendipity than a pure CBF system.
    *   **Improved Accuracy**: Hybrid systems often outperform individual systems by leveraging different signals (user behavior and item content).
*   **Weaknesses**:
    *   **Increased Complexity**: The system is more complex to build and tune. Finding the optimal weights (`cf_weight`, `cbf_weight`) can be challenging and

## Task 3

#### start preprocessing text,extract features with 2 methods and train the 3 models


In [8]:
import glob

def convert_rating_to_label(rating):
    # Convert to 0-100 scale for cleaner integer comparisons
    rating_int = int(rating * 100)
    if rating_int >= 70:
        return "positive"
    elif rating_int <= 30:
        return "negative"
    else:
        return "neutral"

#df['label'] = df['rating'].apply(convert_rating_to_label)

def document_to_vector(text, model, num_features):
    words = text.split()
    #initialize a zero vector for the document.
    feature_vec = np.zeros((num_features,), dtype="float32")
    n_words = 0
    for word in words:
        if word in model:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    #avoid division by zero if no words from the document were found in the model
    if n_words > 0:
        feature_vec = np.divide(feature_vec, n_words)

    return feature_vec

# whole_df=csv_dict["Software_reviews.csv"]
# whole_df = pd.read_csv(f"{save_path}Software_reviews.csv")
# sent_df = pd.read_csv(f"{save_path}Software_sentiment_score.csv")
# Append sentiment scores from multiple categories (take 10k from each)
dfs = []
sent_dfs = []
CATEGORIES = [
    'All_Beauty',
    'Software',
    'Appliances',
    'Digital_Music'
]
MAX_ROWS_PER_CATEGORY = 10000  # Max rows to process per category
save_path = "TEDE/"
for cat in CATEGORIES:
    df_cat = pd.read_csv(f"{save_path}{cat}_reviews.csv", nrows=MAX_ROWS_PER_CATEGORY)
    sent_cat = pd.read_csv(f"{save_path}{cat}_sentiment_score.csv", nrows=MAX_ROWS_PER_CATEGORY)
    dfs.append(df_cat)
    sent_dfs.append(sent_cat)
whole_df = pd.concat(dfs, ignore_index=True)
sent_df = pd.concat(sent_dfs, ignore_index=True)
# only keep relevant fields if needed, e.g. sent_df = sent_df[['text', 'final_sentiment_score']]
# df = whole_df[['title', 'text', 'rating']].copy()
df = whole_df[['text', 'rating']].copy()
df['label'] = sent_df['final_sentiment_score'].apply(convert_rating_to_label)
# print(f"Total rows in whole_df: {len(df)}")
# print(df.head(10))
# import sys
# sys.exit(0) #exit the program if you want to stop here, otherwise comment this line out
# display(df.head(10))
# import sys
# sys.exit(0) #exit the program if you want to stop here, otherwise comment this line out
#print how many have null rating, just to check
print("Null columns:",df.isnull().sum())
#drop any entries with empty fields
df.dropna(inplace=True)

#apply preprocessing to text
# df['title'] = df['title'].apply(preprocess_text)
df['text'] = df['text'].apply(preprocess_text)



#split ds to train and test
X=df['text']
Y=df['label']
#better to use a numerically encoded version instead of strings
Y_encoded = label_encoder.fit_transform(Y)
X_train_text, X_test_text, Y_train, Y_test = train_test_split(
    X,
    Y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=Y_encoded
)


#declare 10-fold
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
#define metrics we're interested in
scoring_metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

#list to present final results
cross_table=[]

#gather all results to present them for the second table
second_table=[]

#=============================== TF-IDF ====================================

tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2)
)

#fit the vectorizer ONLY on the training data
tfidf_vectorizer.fit(X_train_text)

#apply that to both sets
X_train_tfidf = tfidf_vectorizer.transform(X_train_text)
X_test_tfidf = tfidf_vectorizer.transform(X_test_text)


# Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, Y_train)
y_pred_nb_tfidf = nb_model.predict(X_test_tfidf)
print("Naive-Bayes with TF-IDF")
print(classification_report(Y_test, y_pred_nb_tfidf, target_names=label_encoder.classes_))
result=cross_validate(nb_model, X_train_tfidf, Y_train, cv=k_fold, scoring=scoring_metrics)

#store the average results in a list
cross_table.append(
    {
        'Feature Set': 'TF-IDF',
        'Model': 'Naive Bayes',
        'Accuracy': np.mean(result['test_accuracy']),
        'Precision': np.mean(result['test_precision_macro']),
        'Recall': np.mean(result['test_recall_macro']),
        'F1-Score': np.mean(result['test_f1_macro'])
    }
)

second_table.append(
    {
    'Feature Set': 'TF-IDF',
    'Model': 'Naive Bayes',
    'Accuracy': accuracy_score(Y_test, y_pred_nb_tfidf),
    'Precision': precision_score(Y_test, y_pred_nb_tfidf, average='macro',zero_division=0),
    'Recall': recall_score(Y_test, y_pred_nb_tfidf, average='macro',zero_division=0),
    'F1-Score': f1_score(Y_test, y_pred_nb_tfidf, average='macro',zero_division=0)
    }
)


# KNN
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_tfidf, Y_train)
y_pred_knn_tfidf = knn_model.predict(X_test_tfidf)
print("KNN with TF-IDF")
print(classification_report(Y_test, y_pred_knn_tfidf, target_names=label_encoder.classes_))
result=cross_validate(knn_model, X_train_tfidf, Y_train, cv=k_fold, scoring=scoring_metrics)
cross_table.append(
    {
        'Feature Set': 'TF-IDF',
        'Model': 'KNN',
        'Accuracy': np.mean(result['test_accuracy']),
        'Precision': np.mean(result['test_precision_macro']),
        'Recall': np.mean(result['test_recall_macro']),
        'F1-Score': np.mean(result['test_f1_macro'])
    }
)
second_table.append({
    'Feature Set': 'TF-IDF',
    'Model': 'KNN',
    'Accuracy': accuracy_score(Y_test, y_pred_knn_tfidf),
    'Precision': precision_score(Y_test, y_pred_knn_tfidf, average='macro',zero_division=0),
    'Recall': recall_score(Y_test, y_pred_knn_tfidf, average='macro',zero_division=0),
    'F1-Score': f1_score(Y_test, y_pred_knn_tfidf, average='macro',zero_division=0)
})

# Random Forest                                                       #uses all cores to run faster
rf_model = RandomForestClassifier(n_estimators=100, random_state=42,n_jobs=-1 )
rf_model.fit(X_train_tfidf, Y_train)
y_pred_rf_tfidf = rf_model.predict(X_test_tfidf)
print("Random Forest with TF-IDF")
print(classification_report(Y_test, y_pred_rf_tfidf, target_names=label_encoder.classes_))
result=cross_validate(rf_model, X_train_tfidf, Y_train, cv=k_fold, scoring=scoring_metrics)
cross_table.append(
    {
        'Feature Set': 'TF-IDF',
        'Model': 'Random Forest',
        'Accuracy': np.mean(result['test_accuracy']),
        'Precision': np.mean(result['test_precision_macro']),
        'Recall': np.mean(result['test_recall_macro']),
        'F1-Score': np.mean(result['test_f1_macro'])
    }
)
second_table.append({
    'Feature Set': 'TF-IDF',
    'Model': 'Random Forest',
    'Accuracy': accuracy_score(Y_test, y_pred_rf_tfidf),
    'Precision': precision_score(Y_test, y_pred_rf_tfidf, average='macro',zero_division=0),
    'Recall': recall_score(Y_test, y_pred_rf_tfidf, average='macro',zero_division=0),
    'F1-Score': f1_score(Y_test, y_pred_rf_tfidf, average='macro',zero_division=0)
})


#=============================== Embeddings ====================================


#load embedding model
#option1: word2vec
# model_name = 'word2vec-google-news-300'
#option 2:fasttext,often a good choice due to OOV handling
model_name = 'fasttext-wiki-news-subwords-300'

embedding_model = api.load(model_name)
embedding_dim = embedding_model.vector_size



# apply the embedding to the sets
X_train_embeddings = np.array([document_to_vector(doc, embedding_model, embedding_dim) for doc in X_train_text])
X_test_embeddings = np.array([document_to_vector(doc, embedding_model, embedding_dim) for doc in X_test_text])


#(Gaussian) Naive Bayes
gnb_model = GaussianNB()
gnb_model.fit(X_train_embeddings, Y_train)
y_pred_gnb = gnb_model.predict(X_test_embeddings)
print("Naive Bayes with Word Embeddings")
print(classification_report(Y_test, y_pred_gnb, target_names=label_encoder.classes_))
result=cross_validate(gnb_model, X_train_embeddings, Y_train, cv=k_fold, scoring=scoring_metrics)
cross_table.append(
    {
        'Feature Set': 'Word Embeddings',
        'Model': 'Naive Bayes',
        'Accuracy': np.mean(result['test_accuracy']),
        'Precision': np.mean(result['test_precision_macro']),
        'Recall': np.mean(result['test_recall_macro']),
        'F1-Score': np.mean(result['test_f1_macro'])
    }
)
second_table.append({
    'Feature Set': 'Word Embeddings',
    'Model': 'Naive Bayes',
    'Accuracy': accuracy_score(Y_test, y_pred_gnb),
    'Precision': precision_score(Y_test, y_pred_gnb, average='macro'),
    'Recall': recall_score(Y_test, y_pred_gnb, average='macro'),
    'F1-Score': f1_score(Y_test, y_pred_gnb, average='macro')
})

# KNN
knn_model_emb = KNeighborsClassifier(n_neighbors=5)
knn_model_emb.fit(X_train_embeddings, Y_train)
y_pred_knn_emb = knn_model_emb.predict(X_test_embeddings)
print("KNN with Word Embeddings")
print(classification_report(Y_test, y_pred_knn_emb, target_names=label_encoder.classes_))
result=cross_validate(knn_model_emb, X_train_embeddings, Y_train, cv=k_fold, scoring=scoring_metrics)
cross_table.append(
    {
        'Feature Set': 'Word Embeddings',
        'Model': 'KNN',
        'Accuracy': np.mean(result['test_accuracy']),
        'Precision': np.mean(result['test_precision_macro']),
        'Recall': np.mean(result['test_recall_macro']),
        'F1-Score': np.mean(result['test_f1_macro'])
    }
)
second_table.append({
    'Feature Set': 'Word Embeddings',
    'Model': 'KNN',
    'Accuracy': accuracy_score(Y_test, y_pred_knn_emb),
    'Precision': precision_score(Y_test, y_pred_knn_emb, average='macro',zero_division=0),
    'Recall': recall_score(Y_test, y_pred_knn_emb, average='macro',zero_division=0),
    'F1-Score': f1_score(Y_test, y_pred_knn_emb, average='macro',zero_division=0)
})

# Random Forest
rf_model_emb = RandomForestClassifier(n_estimators=100, random_state=42,n_jobs=-1)
rf_model_emb.fit(X_train_embeddings, Y_train)
y_pred_rf_emb = rf_model_emb.predict(X_test_embeddings)
print("Random Forest with Word Embeddings")
print(classification_report(Y_test, y_pred_rf_emb, target_names=label_encoder.classes_))
result=cross_validate(rf_model_emb, X_train_embeddings, Y_train, cv=k_fold, scoring=scoring_metrics)
cross_table.append(
    {
        'Feature Set': 'Word Embeddings',
        'Model': 'Random Forest',
        'Accuracy': np.mean(result['test_accuracy']),
        'Precision': np.mean(result['test_precision_macro']),
        'Recall': np.mean(result['test_recall_macro']),
        'F1-Score': np.mean(result['test_f1_macro'])
    }
)
second_table.append({
    'Feature Set': 'Word Embeddings',
    'Model': 'Random Forest',
    'Accuracy': accuracy_score(Y_test, y_pred_rf_emb),
    'Precision': precision_score(Y_test, y_pred_rf_emb, average='macro',zero_division=0),
    'Recall': recall_score(Y_test, y_pred_rf_emb, average='macro',zero_division=0),
    'F1-Score': f1_score(Y_test, y_pred_rf_emb, average='macro',zero_division=0)
})

#print results of 10-fold cross validation
cross_table_df = pd.DataFrame(cross_table)
#display the results table for the cross validation
print("--- 10-Fold Cross Validation Performance ---")



display(cross_table_df.round(4))

print("\n\n")


#convert to dataframe
second_table_df = pd.DataFrame(second_table)

#display the results table for the test set
print("--- Final Test Set Performance ---")
display(second_table_df.round(4))
cross_table_df.to_csv(os.path.join(save_path, "cross_validation_performance.csv"), index=False, float_format='%.4f')
second_table_df.to_csv(os.path.join(save_path, "test_set_performance.csv"), index=False, float_format='%.4f')



Null columns: text      4
rating    0
label     0
dtype: int64
Naive-Bayes with TF-IDF
              precision    recall  f1-score   support

    negative       0.71      0.19      0.30       932
     neutral       0.48      0.04      0.08      1194
    positive       0.76      0.99      0.86      5874

    accuracy                           0.76      8000
   macro avg       0.65      0.41      0.41      8000
weighted avg       0.71      0.76      0.68      8000

KNN with TF-IDF
              precision    recall  f1-score   support

    negative       0.30      0.10      0.15       932
     neutral       0.20      0.10      0.13      1194
    positive       0.75      0.91      0.82      5874

    accuracy                           0.69      8000
   macro avg       0.42      0.37      0.37      8000
weighted avg       0.62      0.69      0.64      8000

Random Forest with TF-IDF
              precision    recall  f1-score   support

    negative       0.61      0.28      0.38       932


,Feature Set,Model,Accuracy,Precision,Recall,F1-Score
0,TF-IDF,Naive Bayes,0.7555,0.6536,0.4071,0.4117
1,TF-IDF,KNN,0.6856,0.4023,0.3647,0.3628
2,TF-IDF,Random Forest,0.7635,0.6361,0.4441,0.4608
3,Word Embeddings,Naive Bayes,0.4313,0.4498,0.4656,0.3773
4,Word Embeddings,KNN,0.7153,0.4882,0.4239,0.4376
5,Word Embeddings,Random Forest,0.7410,0.6026,0.3629,0.3409





--- Final Test Set Performance ---


,Feature Set,Model,Accuracy,Precision,Recall,F1-Score
0,TF-IDF,Naive Bayes,0.7564,0.6496,0.4084,0.4136
1,TF-IDF,KNN,0.6916,0.4166,0.3692,0.3689
2,TF-IDF,Random Forest,0.7624,0.6246,0.4449,0.4610
3,Word Embeddings,Naive Bayes,0.4375,0.4529,0.4697,0.3823
4,Word Embeddings,KNN,0.7116,0.4754,0.4167,0.4285
5,Word Embeddings,Random Forest,0.7415,0.5722,0.3654,0.3455


### 1. Feature Set Comparison:
#### TF-IDF had
*   **Superior Performance:** For every single algorithm (Naive Bayes, KNN, Random Forest), the **TF-IDF feature set consistently and significantly outperforms Word Embeddings**. This is evident in accuracy, precision, recall, and F1-score in both the cross-validation and final test set results.
*   **Naive Bayes Failure with Embeddings:** The most dramatic failure is the `Naive Bayes with Word Embeddings` model, which has an accuracy of only ~44%. This is expected. Naive Bayes assumes feature independence, a condition that is fundamentally violated by the dense, correlated vectors produced by word embeddings. In contrast, it performs surprisingly well with TF-IDF's sparse, count-based features.
#### Why TF-IDF Worked Better:
* For traditional machine learning models like these, the explicit word frequency information captured by TF-IDF appears to be a much stronger signal for sentiment than the semantic relationships captured by the (likely averaged) word embeddings. Word embeddings typically require more complex models (like neural networks) to effectively leverage their contextual power.